# 웹스크래핑(Web Scrapping)

## 1. 개념

* 웹스크래핑(Web Scrapping) : 웹사이트상에서 특정부분에 위치한 정보를 컴퓨터로 자동추출하는 기능
* 웹크롤링(Web Crawling) : 자동화봇이 정해진 규칙에 따라 복수개의 웹페이지를 브라우징하는 기능

## 2. 필요한 라이브러리

* `BeautifulSoup` : HTML과 XML문선를 파싱하기 위한 라이브러리
* scrapy : Python을 작성된 오픈소스 웹크롤링 프레임워크

## 3. 스크래핑(클롤링)하는 방법

1. 원하는 페이지에 `요청(request)`을 보낸 후 그 결과를 html로 받는다.
1. 받은 html을 `파싱(Parsing)`처리를 한다.
1. 필요한 정보를 추출한다.
1. 파이썬으로 스크래핑을 하기 위해서는 `http request/response 모듈`과 `html을 파싱하는 모듈`이 필요하다.

##### 참고 사이트
* https://www.crummy.com/software/BeautifulSoup
* https://docs.python.org/3.0/library/urllib.request.html

### 1. 웹스크래핑 -  html 소스읽기

In [5]:
# 1. html 소스읽기
from urllib.request import urlopen
html = urlopen('http://www.google.com')
print(type(html), html)
print(dir(html))
print()

html.read()

<class 'http.client.HTTPResponse'> <http.client.HTTPResponse object at 0x000001F811F9F130>
['__abstractmethods__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_abc_impl', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', '_check_close', '_close_conn', '_get_chunk_left', '_method', '_peek_chunked', '_read1_chunked', '_read_and_discard_trailer', '_read_chunked', '_read_next_chunk_size', '_read_status', '_readinto_chunked', '_safe_read', '_safe_readinto', 'begin', 'chunk_left', 'chunked', 'close', 'closed', 'code', 'debuglevel', 'detach', 'fileno', 'flush', 'fp', 'getcode', 'getheader', 'getheaders', 'geturl', 'he

b'<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="ko"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"><title>Google</title><script nonce="72z9NtZl0cOhq7oQgtkRmQ">(function(){var _g={kEI:\'nJtEZYeXD7je2roPx6-lkAc\',kEXPI:\'0,1365468,206,4804,2316,383,246,5,1129120,1197792,380699,16114,28684,22430,1362,12316,17583,4998,17075,38444,885,1987,2891,11754,606,29880,788,30022,16105,230,20583,4,64054,22613,6624,7596,1,42154,2,20270,17733,1758,6699,31123,4567,6256,23421,1252,33064,2,2,1,10956,13670,2006,8155,23351,872,6578,13056,7,1922,9779,42459,20198,6050,14087,14,82,20206,8377,24363,782,2248,15816,1804,13806,13288,7899,276,11813,1635,3241,6467,6058,5736,12965,146,9629,1811,5213516,6739,190,2,762,5994203,2803214,3311,141,795,29675,281,23940611,579,4043528,16673,37284,2173,4429,3,1558,5,542,3,1393942,23759270,6870,2,5927,8408,2880,1594,9048,1178,729

In [12]:
# 2. 예외처리방법
from urllib.error import HTTPError
from urllib.error import URLError

try:
    html = urlopen('http://www.naverxx.com')
except HTTPError as e:
    print('HTTP 에러발생!!')
except URLError as e:    
    print('존재하지 않는 사이트 주소입니다!')
else:
    print(html.read())

존재하지 않는 사이트 주소입니다!


In [14]:
# 3. 이미지다운로드방법(1) - 간편한 방법
import urllib.request

# 다음사이트의 로고이미지를 다운로드하기
# https://t1.daumcdn.net/daumtop_chanel/op/20200723055344399.png
url = 'https://t1.daumcdn.net/daumtop_chanel/op/20200723055344399.png'
save_file = './data/daum.png'

urllib.request.urlretrieve(url, save_file)
print('파일이 정상적으로 다운로드 되었습니다!!')

파일이 정상적으로 다운로드 되었습니다!!


In [17]:
# 3. 이미지다운로드방법(2) - 바이너리파일로 처리
image = urllib.request.urlopen(url).read()
# print(type(image), image)

# 바이너리로 저장
with open('./data/daum_1.png', mode='wb') as f:
    f.write(image)
    print('파일이 정상적으로 다운로드 되었습니다!!')

파일이 정상적으로 다운로드 되었습니다!!


In [4]:
# 4. 매개변수를 추가해서 인터넷의 자료를 요청하기
# 기상청의 일기예보 사이트 : http://www.kma.go.kr
# 기상청육상중기예보
import urllib
from urllib.request import urlopen

API = 'https://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp'

# url에 특수문자, 한글이 포함된 경우에는 URL인코딩이 필요
# 지역번호
#   전국 108, 서울/경기 109, 강원 105, 충북 131, 충남 133, 전북 146
#   전남 156, 경북 143, 경남 159, 제주 184

values = {'stnId':'108'} # 전국 날씨예보

# url인코딩
params = urllib.parse.urlencode(values)
print(type(params), params)

# 요청할 URL을 생성
url = API + '?' + params
print(url)

# 응답된 자료 읽기
data = urllib.request.urlopen(url).read() # 바이트타입으로 리턴
#  print(type(data), data)

# response데이터를 decoding
text = data.decode('utf-8')
print(text)

<class 'str'> stnId=108
https://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=108
<?xml version="1.0" encoding="utf-8" ?>
<rss version="2.0">
<channel>
<title>기상청 육상 중기예보</title>
<link>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp</link>
<description>기상청 날씨 웹서비스</description>
<language>ko</language>
<generator>기상청</generator>
<pubDate>2023년 11월 06일 (월)요일 06:00</pubDate>
 <item>
<author>기상청</author>
<category>육상중기예보</category>
<title>전국 육상 중기예보 - 2023년 11월 06일 (월)요일 06:00 발표</title>
<link>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp</link>
<guid>http://www.kma.go.kr/weather/forecast/mid-term_01.jsp</guid>
<description>
	<header>
		<title>전국 육상중기예보</title>
		<tm>202311060600</tm>
		<wf><![CDATA[○ (하늘상태, 강수) 이번 예보기간에는 전국이 가끔 구름많겠으나, 10일(금) 오전 제주도에 비가 오겠습니다. <br />○ (기온) 9일(목)~10일(금) 아침 기온은 4~15도, 낮 기온은 10~21도로 평년(최저기온 0~10도, 최고기온 11~18도)보다 조금 높겠으며,<br />          11일(토)~16일(목) 아침 기온은 -4~9도, 낮 기온은 7~17도로 평년과 비슷하거나 조금 낮겠습니다.<br />○ (주말전망) 11일(토)은 오전에 구름많다가 오후에 

## 2. 웹스크래핑 - BeautifulSoup

* 다운로드 : https://www.crummy.com/software/BeautifulSoup
* `pip install beautifulsoup4` or `conda install -y beautifulsoup4`
* 주의) ~4를 생략하면 3.x버전이 설치 된다.

In [6]:
!pip show beautifulsoup4

Name: beautifulsoup4
Version: 4.12.2
Summary: Screen-scraping library
Home-page: 
Author: 
Author-email: Leonard Richardson <leonardr@segfault.org>
License: 
Location: C:\Anaconda3\Lib\site-packages
Requires: soupsieve
Required-by: conda-build, nbconvert


In [14]:
# 1. BeautifulSoup : HTML을 파싱하는 Library
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs

# 1) url request
# https://www.google.com의 메이페이지의 타이틀 텍스트 가져오기
# url에 접속요청이 성공이 되면 HTTPResponse객체가 리턴된다.
html = urlopen('https://www.google.com')
print(type(html), html)

# 2) html.read() : HTTPResponse에서 html 데이터를 읽기
# BS의 기본분석기를 이용하여 html을 분석하기 위한 객체를 생성
# html분석기 -> html, xml분석기 -> xml
# html.read()
soup = bs(html.read(), 'html.parser')
# print(type(soup), soup)

# 문서내의 첫 h1태그를 선택
print(soup.h1)

<class 'http.client.HTTPResponse'> <http.client.HTTPResponse object at 0x000001FF5BA1F6A0>
None


In [28]:
# 2. next_sibling

# 분석하고자하는 html
html ='''
<html><body>
    <h1>Hello Web Scraping</h1>
    <p>웹 페이지 분석</p>
    <p>웹 스크래핑</p>
</body></html>
'''
soup = bs(html, 'html.parser')
print(soup.h1, soup.p)

h1 = soup.html.body.h1
p1 = soup.html.body.p
print(h1, p1, type(p1))

# sibling 형제노드(동일레벨의 노드)
# previous_sibiling : 동일레벨의 이전노드
# next_sibling      : 동일레벨의 다음노드

p2 = p1.previous_sibling.previous_sibling 
# 2번하는 이유는 h1, \n
# 즉,  p1.previous_sibling -> \n(줄바꿈 문자) 선택
#      p1.previous_sibling.previous_sibling  -> h1 tag선택
print(p2)

p2 = p1.next_sibling   # 줄바꿈문자선책
p2 = p1.next_sibling.next_sibling # 2번째 p태그선택
print(p2)
print()

# Tag의 내부 text 추출하기
print(f'h1 = {h1.string}')
print(f'p1 = {p1.string}')
print(f'p2 = {p2.string}')

<h1>Hello Web Scraping</h1> <p>웹 페이지 분석</p>
<h1>Hello Web Scraping</h1> <p>웹 페이지 분석</p> <class 'bs4.element.Tag'>
<h1>Hello Web Scraping</h1>
<p>웹 스크래핑</p>

h1 = Hello Web Scraping
p1 = 웹 페이지 분석
p2 = 웹 스크래핑


In [35]:
# 3. find()
html ='''
<html><body>
    <h1 id="title">Hello Web Scraping</h1>
    <p id="body">웹 페이지 분석</p>
    <p>웹 스크래핑</p>
</body></html>
'''

soup = bs(html, 'html.parser')
# print(dir(soup))
# soup.find?
# 실습. id title인 태그, id가 body인 태그를 선택하고
# 내부문자열을 출력해보기
title = soup.find(id='title')
body = soup.find(id='body')
print(title, body)
print()

print(f'title = {title.string}')
print(f'body = {body.string}')

<h1 id="title">Hello Web Scraping</h1> <p id="body">웹 페이지 분석</p>

title = Hello Web Scraping
body = 웹 페이지 분석


In [42]:
# 4. find_all() : 특정속성을 가진 tag 전부를 추출하기
html ='''
<html><body>
    <li><a href="http://daum.net">daum</a></li>
    <li><a href="http://google.com">google</a></li>
    <li><a href="http://yahoo.com">yahoo</a></li>
    <li><a href="http://nate.com">nate</a></li>
    <li><a href="http://naver.com">naver</a></li>
</body></html>
'''

# 실습2. a태그 전체를 추출하기
# 전체를 가져올경우 데이터의 타입확인(bs4.element.ResultSet) 
# list자료형과 동일, list는 for문을 이용해서 출력

soup = bs(html, 'html.parser')

links = soup.find_all('a')
print(type(links), links)
print()

for link in links:
    # print(type(link), link)
    text = link.string
    # 태그내부의 속성 가져오기 : attrs
    href = link.attrs['href']
    print(f'{text} = {href}')

<class 'bs4.element.ResultSet'> [<a href="http://daum.net">daum</a>, <a href="http://google.com">google</a>, <a href="http://yahoo.com">yahoo</a>, <a href="http://nate.com">nate</a>, <a href="http://naver.com">naver</a>]

daum = http://daum.net
google = http://google.com
yahoo = http://yahoo.com
nate = http://nate.com
naver = http://naver.com


In [48]:
# 5. CSS처리 : select_one(), select()
# 실습. h1의 도서목록을 추출하기 -> select_one() 
#       ... css선택자 : div#main > h1 
#       li목록 -> select()
#       ... css선택자 : div#main > ul > li
html ='''
<html><body>
    <div id="main">
        <h1>도서목록</h1>
        <ul>
            <li>자바프로그램 입문</li>
            <li>파이썬으로 하는 데이터분석</li>
            <li>HTML5/CSS3</li>
        </ul>
    </div>
</body></html>
'''
soup = bs(html, 'html.parser')
# soup.select?
# soup.select_one?

h1 = soup.select_one('div#main > h1').string
print(soup.h1.string, '=', h1)

# li목록 : select()
li_list = soup.select('div#main > ul > li')
print(type(li_list), li_list)
print()

for li in li_list:
    print(f'li = {li.string}')

도서목록 = 도서목록
<class 'bs4.element.ResultSet'> [<li>자바프로그램 입문</li>, <li>파이썬으로 하는 데이터분석</li>, <li>HTML5/CSS3</li>]

li = 자바프로그램 입문
li = 파이썬으로 하는 데이터분석
li = HTML5/CSS3


#### 실습1. 기상청의 일기예보
![CDATA[○ (하늘상태, 강수) 이번 예보기간에는 전국이 가끔 구름많겠으나, 10일(금) 오전 제주도에 비가 오겠습니다. <br />○ (기온) 9일(목)~10일(금) 아침 기온은 4~15도, 낮 기온은 10~21도로 평년(최저기온 0~10도, 최고기온 11~18도)보다 조금 높겠으며,<br />          11일(토)~16일(목) 아침 기온은 -4~9도, 낮 기온은 7~17도로 평년과 비슷하거나 조금 낮겠습니다.<br />○ (주말전망) 11일(토)은 오전에 구름많다가 오후에 대체로 맑겠으나 강원영동과 제주도는 대체로 흐리겠습니다. 12일(일)은 중부지방은 대체로 맑겠고, 남부지방은 구름많겠습니다.<br />              아침 기온은 -2~9도, 낮 기온은 7~17도가 되겠습니다.]]

In [58]:
import urllib.request as req 
from bs4 import BeautifulSoup as bs

stnId = input('지역번호를 입력하세요 => ')
url = f'https://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId={stnId}'
print(url)

res = req.urlopen(url)
soup = bs(res, 'html.parser')

title = soup.find('title').string
print(title)
print()

wf = soup.find('wf').string
wf = wf.replace('<br />', '')
print(wf)

지역번호를 입력하세요 => 109
https://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=109
기상청 육상 중기예보

○ (하늘상태) 이번 예보기간 동안 대체로 맑거나 구름많은 날이 많겠습니다.○ (기온) 9일(목)~11일(토) 아침기온 0~10도, 낮 기온 8~20도로 평년(최저기온 1~7도, 최고기온 13~15도)과 비슷하거나 조금 높겠으며,         12일(일)~16일(목)은 아침기온 -5~6도, 낮 기온 6~12도로 평년(최저기온 -2~6도, 최고기온 10~14도)보다 낮아 춥겠으니, 건강관리에 유의하기 바랍니다.○ (해상) 서해중부해상의 물결은 10일(금)~11일(토) 1.0~3.0m로 높게 일겠고, 그 밖의 날은 0.5~2.5m로 일겠습니다.○ (주말전망) 11일(토)~12일(일)은 대체로 맑겠으며, 아침 기온은 -4~2도, 낮 기온은 6~10도가 되겠습니다.


#### 실습2. 네이버금융에서 환율정보 추출하기

* http://finance.naver.com/marketindex
* 출력 usd/krw = 1302.20

In [62]:
url = 'http://finance.naver.com/marketindex'
res = req.urlopen(url)
soup = bs(res, 'html.parser')
# <div class="head_info point_dn">
# <span class="value">1,302.20</span>

ex_rate = soup.select_one('div.head_info > span.value')
# print(ex_rate)
print(f'usd/krw = {ex_rate.string}')

usd/krw = 1,301.10


In [66]:
ex_rates = soup.select('div.head_info > span.value')
print(type(ex_rate), ex_rate)
print()
for rate in ex_rates:
    print(rate.string) # 통화별 환율구분해 보기

<class 'bs4.element.ResultSet'> [<span class="value">1,301.10</span>, <span class="value">869.57</span>, <span class="value">1,396.34</span>, <span class="value">178.52</span>, <span class="value">149.3500</span>, <span class="value">1.0736</span>, <span class="value">1.2375</span>, <span class="value">104.8500</span>, <span class="value">80.51</span>, <span class="value">1727.65</span>, <span class="value">1999.2</span>, <span class="value">83070.58</span>]

1,301.10
869.57
1,396.34
178.52
149.3500
1.0736
1.2375
104.8500
80.51
1727.65
1999.2
83070.58


#### 실습3. 윤동주시인의 작품목록 추출하기

* https://ko.wikisource.org/wiki/저자:윤동주
  -- 한글주소상태로 복사하기 - 크롬웹스토어에서 `copy unicode urls` 설치
* select() 
* 웹브라우저에서 F12개발자도구, elements에서 원하는 부분을 우클릭
  - Copy > Copy selector : CSS 셀렉터

In [73]:
url = 'https://ko.wikisource.org/wiki/%EC%A0%80%EC%9E%90:%EC%9C%A4%EB%8F%99%EC%A3%BC'
res = req.urlopen(url)
soup = bs(res, 'html.parser')

a_list = soup.select('.mw-parser-output > ul > li > a')
# print(type(a_list), len(a_list), a_list)

for a in a_list:
    name = a.string
    print(f'... {name}')


... 증보판
... 흰 그림자
... 사랑스런 추억
... 흐르는 거리
... 봄
... 참회록
... 간(肝)
... 위로
... 팔복
... 못자는밤
... 달같이
... 고추밭
... 아우의 인상화
... 사랑의 전당
... 이적
... 비오는 밤
... 산골물
... 유언
... 창
... 바다
... 비로봉
... 산협의 오후
... 명상
... 소낙비
... 한난계
... 풍경
... 달밤
... 장
... 밤
... 황혼이 바다가 되어
... 아침
... 빨래
... 꿈은 깨어지고
... 산림
... 이런날
... 산상
... 양지쪽
... 닭
... 가슴 1
... 가슴 2
... 비둘기
... 황혼
... 남쪽 하늘
... 창공
... 거리에서
... 삶과 죽음
... 초한대
... 산울림
... 해바라기 얼굴
... 귀뚜라미와 나와
... 애기의 새벽
... 햇빛·바람
... 반디불
... 둘 다
... 거짓부리
... 눈
... 참새
... 버선본
... 편지
... 봄
... 무얼 먹구 사나
... 굴뚝
... 햇비
... 빗자루
... 기왓장 내외
... 오줌싸개 지도
... 병아리
... 조개껍질
... 겨울
... 트루게네프의 언덕
... 달을 쏘다
... 별똥 떨어진 데
... 화원에 꽃이 핀다
... 종시


### 3. BeautifulSoup 관련함수

<!DOCTYPE html>
<html>
<head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>
<p class="story">
    Once upon a time there were three little sisters; and their names were
    <a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
    <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
    <a href="http://example.com/tillie" class="sister brother" id="link3">Tillie</a>;
    and they lived at the bottom of a well.
</p>
</body>
</html>

In [74]:
import urllib.request as req 
from bs4 import BeautifulSoup as bs

In [115]:
html = '''
    <!DOCTYPE html>
    <html>
    <head><title>The Dormouse's story</title></head>
    <body>
    <p class="title"><b>The Dormouse's story</b></p>
    <p class="story">
        Once upon a time there were three little sisters; and their names were
        <a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
        <a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
        <a href="http://example.com/tillie" class="sister brother" id="link3">Tillie</a>;
        and they lived at the bottom of a well.
    </p>
    </body>
    </html>
'''
soup = bs(html, 'html.parser')

# 1. element
print(soup.title, soup.find('title'))

# 2. tag 이름
print(soup.title.name, soup.find('title').name)

# 3. text
print(soup.title.string, soup.find('title').string, soup.title.get_text())

# 4. single element
print(soup.a.get_text(), soup.find('a').string, soup.find('a').get_text())

# 5. multi element
print(soup.find_all('a'))
print(soup.find_all('a')[0].get_text(),soup.find_all('a')[1].get_text(),soup.find_all('a')[2].get_text())

# 6. attribute
print(soup.a['class'], soup.a.get('class'))
print(soup.a['href'], soup.a.get('href'))

# 7. find by id
print(soup.find(id='link1'))
print(soup.find('a', {'id':'link1'}))
print()

# 8. find by class
print(soup.find(class_='sister'))
print(soup.find('a', {'class':'sister'}))
print(soup.find_all('a', {'class':'sister'}))

<title>The Dormouse's story</title> <title>The Dormouse's story</title>
title title
The Dormouse's story The Dormouse's story The Dormouse's story
Elsie Elsie Elsie
[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, <a class="sister brother" href="http://example.com/tillie" id="link3">Tillie</a>]
Elsie Lacie Tillie
['sister'] ['sister']
http://example.com/elsie http://example.com/elsie
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>

<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>
<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>
[<a class="sister" href="http://example.com/elsie" id="link1">Elsie</a>, <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>, <a class="sister brother" href="http://example.com/tillie" id="link3">Tillie</a>]


#### 4. 고급 - HTML 분석하기

* https://github.com/REMitchell/python-scraping

In [138]:
# 1. 전쟁과 평화
html = req.urlopen('http://pythonscraping.com/pages/warandpeace.html')
soup = bs(html, 'html.parser')

In [139]:
# 실습 1) span태그에서 class가 green인 태그 : 등장인물의 이름을 출력
# findAll(), find_all()
names = soup.findAll('span', {'class':'green'})
for name in names:
    print(name.get_text())

Anna
Pavlovna Scherer
Empress Marya
Fedorovna
Prince Vasili Kuragin
Anna Pavlovna
St. Petersburg
the prince
Anna Pavlovna
Anna Pavlovna
the prince
the prince
the prince
Prince Vasili
Anna Pavlovna
Anna Pavlovna
the prince
Wintzingerode
King of Prussia
le Vicomte de Mortemart
Montmorencys
Rohans
Abbe Morio
the Emperor
the prince
Prince Vasili
Dowager Empress Marya Fedorovna
the baron
Anna Pavlovna
the Empress
the Empress
Anna Pavlovna's
Her Majesty
Baron
Funke
The prince
Anna
Pavlovna
the Empress
The prince
Anatole
the prince
The prince
Anna
Pavlovna
Anna Pavlovna


In [140]:
# 실습 2) 웹페이지에서 모든 heading태그(h1~h6)의 text를 출력
#         - list안에 for문을 사용
titles = soup.find_all(['h1','h2','h3','h4','h5','h6',])
print([title for title in titles])
print([title.string for title in titles])
print([title.get_text() for title in titles])

[<h1>War and Peace</h1>, <h2>Chapter 1</h2>]
['War and Peace', 'Chapter 1']
['War and Peace', 'Chapter 1']


In [141]:
# 실습 3) span태그에서 class가 green or red인 태그의 text출력
#         - find_all('', {'class': {집합}), list안에 for문
all_text = soup.find_all('span', {'class': ['green', 'red']})
print([text.string for text in all_text])

["Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don't tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by\nthat Antichrist- I really believe he is Antichrist- I will have\nnothing more to do with you and you are no longer my friend, no longer\nmy 'faithful slave,' as you call yourself! But how do you do? I see\nI have frightened you- sit down and tell me all the news.", 'Anna\nPavlovna Scherer', 'Empress Marya\nFedorovna', 'Prince Vasili Kuragin', 'Anna Pavlovna', 'St. Petersburg', 'If you have nothing better to do, Count [or Prince], and if the\nprospect of spending an evening with a poor invalid is not too\nterrible, I shall be very charmed to see you tonight between 7 and 10-\nAnnette Scherer.', 'Heavens! what a virulent attack!', 'the prince', 'Anna Pavlovna', "First of all, dear friend, tell me how you are. Set your friend's\nmind at rest,", 'Can one be well while suffering mora

In [146]:
# 실습 4) wordcount = 'the prince'단어의 갯수를 출력
wordcount = soup.find_all(text='the prince')
print(type(wordcount), len(wordcount), wordcount)
print(f'the prince의 출현갯수 = {len(wordcount)}')

<class 'bs4.element.ResultSet'> 0 []
the prince의 출현갯수 = 0


C:\Users\gilbaek\AppData\Local\Temp\ipykernel_5944\2801849267.py:2: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  wordcount = soup.find_all(text='the prince')


In [147]:
# 자식노드 추출하기
html = req.urlopen('http://pythonscraping.com/pages/page3.html')
soup = bs(html, 'html.parser')

In [153]:
# 실습 5) table태그중에서 id가 giftList인 태그의 자식노드를 추출
# hint) 자식노드 : soup.find().children, soup.find().tr.previous_sibling, next...
a = soup.find('table', {'id':'giftList'}).children
print(type(a))

for child in soup.find('table', {'id':'giftList'}).children:
    print(child)

<class 'list_iterator'>


<tr><th>
Item Title
</th><th>
Description
</th><th>
Cost
</th><th>
Image
</th></tr>


<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src=".

In [154]:
# 실습 6) next_siblings
# table태그에 id giftList인 태그의 자식노드 추출
# hint) soup.find().tr.next_siblings
for sibling in soup.find('table', {'id':'giftList'}).tr.next_siblings:
    print(sibling)



<tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>


<tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>


<tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>


<tr class="gift" id="gift4"><td>
Dead Parrot
</td><td>
This is an ex-parr

In [158]:
# 실습 7) previous_sibling
# img1.jpg의 부모노드의 이전형제노드의 text추출하기
# sound.find().parent.previous_sibling을 이용
print(soup.find('img', {'src':'../img/gifts/img1.jpg'}))
print(soup.find('img', {'src':'../img/gifts/img1.jpg'}).parent.previous_sibling)
print(soup.find('img', {'src':'../img/gifts/img1.jpg'}).parent.previous_sibling.string)
print(soup.find('img', {'src':'../img/gifts/img1.jpg'}).parent.previous_sibling.get_text())

<img src="../img/gifts/img1.jpg"/>
<td>
$15.00
</td>

$15.00


$15.00



In [168]:
# 실습8) 정규식을 사용 - img태그 전체를 추출하기
# re.complie(../img/gifts/img*.jpg)
import re
imgs = soup.find_all('img', {'src': re.compile('\.\.\/img\/gifts\/img.*\.jpg')})

for img in imgs:
    print(img['src'])

../img/gifts/img1.jpg
../img/gifts/img2.jpg
../img/gifts/img3.jpg
../img/gifts/img4.jpg
../img/gifts/img6.jpg


In [173]:
# 실습 9) lambda식이용

# a. tag속성이 2개인 것만 추출
print(soup.find_all(lambda tag: len(tag.attrs)>=2))
print()

# b. text = 'Or maybe he\'s only resting?'만 추출
print(soup.find_all(lambda tag: tag.get_text() == 'Or maybe he\'s only resting?'))
print(soup.find_all('', text='Or maybe he\'s only resting?'))

[<img src="../img/gifts/logo.jpg" style="float:left;"/>, <tr class="gift" id="gift1"><td>
Vegetable Basket
</td><td>
This vegetable basket is the perfect gift for your health conscious (or overweight) friends!
<span class="excitingNote">Now with super-colorful bell peppers!</span>
</td><td>
$15.00
</td><td>
<img src="../img/gifts/img1.jpg"/>
</td></tr>, <tr class="gift" id="gift2"><td>
Russian Nesting Dolls
</td><td>
Hand-painted by trained monkeys, these exquisite dolls are priceless! And by "priceless," we mean "extremely expensive"! <span class="excitingNote">8 entire dolls per set! Octuple the presents!</span>
</td><td>
$10,000.52
</td><td>
<img src="../img/gifts/img2.jpg"/>
</td></tr>, <tr class="gift" id="gift3"><td>
Fish Painting
</td><td>
If something seems fishy about this painting, it's because it's a fish! <span class="excitingNote">Also hand-painted by trained monkeys!</span>
</td><td>
$10,005.00
</td><td>
<img src="../img/gifts/img3.jpg"/>
</td></tr>, <tr class="gift" id="

C:\Users\gilbaek\AppData\Local\Temp\ipykernel_5944\443448495.py:9: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  print(soup.find_all('', text='Or maybe he\'s only resting?'))


### 5. json분석

In [181]:
# json탐색 : {key:value}
import json
json_str = '''{
    "amount":[{"num":0},{"num":1},{"num":2}], 
    "fruits":[{"fruit":"apple"},{"fruit":"banana"},{"fruit":"pear"}]
}'''
print(type(json_str), json_str)

# json을 dict 변환
dict_data = json.loads(json_str) 
print(type(dict_data), dict_data)
print()

print(type(dict_data.get('fruits')), dict_data.get('fruits'))
print(dict_data.get('fruits')[0])
print(dict_data.get('fruits')[1])
print(dict_data.get('fruits')[2])

<class 'str'> {
    "amount":[{"num":0},{"num":1},{"num":2}], 
    "fruits":[{"fruit":"apple"},{"fruit":"banana"},{"fruit":"pear"}]
}
<class 'dict'> {'amount': [{'num': 0}, {'num': 1}, {'num': 2}], 'fruits': [{'fruit': 'apple'}, {'fruit': 'banana'}, {'fruit': 'pear'}]}

<class 'list'> [{'fruit': 'apple'}, {'fruit': 'banana'}, {'fruit': 'pear'}]
{'fruit': 'apple'}
{'fruit': 'banana'}
{'fruit': 'pear'}


### 6. pdf 분석

* `pip install pdfminer3k`

In [182]:
!pip install pdfminer3k

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/100.8 kB ? eta -:--:--
     ---- ----------------------------------- 10.2/100.8 kB ? eta -:--:--
     ---------------------- -------------- 61.4/100.8 kB 812.7 kB/s eta 0:00:01
     -------------------------------------- 100.8/100.8 kB 1.2 MB/s eta 0:00:00


['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__length_hint__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__']
